# NightCrawler

Patrizio Di Fraia

Ion Chiriac

Andrea D'Antonio *

Aref Ghazouani *

# Crowd-Powered Find Algorithms



Il campo del crowdsourcing, combina le capacità computazionali delle macchine e dell'uomo per risolvere problemi che potrebbero essere difficili ai due se presi singolarmente.
Uno scenario tipico è quello dell'uomo che individua degli elementi all'interno di immagini o video, operazione che non risulta seplice da eseguire per un calcolatore elettronico.

L'algoritmo di CrowFind è uno strumento che si abbina al crowdsourcing per determinare il giusto numero di domande da fare e come farle.

ALGORITMO BASE DI CROWDFIND : 
dato un numero di oggetti una proprietà booleana P, un numero K, attraverso domande fatte agli umani si vogliono trovare K oggetti che soddisfano P. 

E' importante precisare che non esiste un'unica soluzione ne una soluzione che sia la migliore,  ci sono invece varie implementazioni che tendono conto del tradeoff tra costo e tempo.

Scenario sequenziale: viene fatta una domanda per volta finchè non si arriva a K oggetti che soddisfano P. Costo minimo ma tempo elevato.

Scenario parallelo: vengono mostrate tutte le immagini in parallelo in unica fase. Tempo minimo ma costo massimo.

Esistono inoltre scenari ibridi in cui si possono settare delle soglie entro cui potersi muovere o in termine di fasi o di domande.

Tutto ciò tiene conto di due possbili scenari: quando l'umano non sbaglia mai(ideale ma irreale), e quando l'uomo può sbagliare nel rispondere.

Per stabilire in questo secondo caso se un oggetto gode o meno della proprietà P,  vengono fatte più domande sullo stesso oggetto, e vengono utilizzate  delle strategie per determinare il risultato (nel paper Rectangular, Majority e Treshold). 
Ogni punto della strategia viene individuato dalla coppia (n1,n2)  dove n1= numero di risposte Yes per l'oggetto  e n2= numero di risposte No.
Per ognuno di questi punti viene calcolato il costo aspettato Y(n1,n2), che serve per scegliere se per trovare il prossimo oggetto che soddisfa la proprietà conviene continuare a fare domande su quell'oggetto oppure scartarlo e passare ad esaminarne un altro.
La Y tiene conto della selettività a priori e dell'errore rate che si considera essere uguale per ogni utente.






# Algoritmo UncOptCost



```
Data: I, K1 , Pr(⋅), strategy S
Result: Set L of K 1 1 items
L ← ∅;
U = I;
Compute Y for each point in the strategy R;
while ∣L∣ < K 1 do
    Pick I ′ ⊆ U , ∣I ′ ∣ = (K 1 − ∣L∣) items with the lowest Y (R(I));
    CQ ← {};
    for each I ∈ I ′ do
        Let n+ be the fewest YES responses required to make Y (R(I)) = 0;
        Let n− be the fewest NO responses required to make
        Y (R(I)) = Y (0, 0);
        Add min{n + , n − } questions on I to CQ;
    Ask CQ in parallel to the crowdsourcing service;
    Update R(I) for each I ∈ CQ based on answers;
    for each I ∈ CQ do
        if S(R(I)) = 1 then
            Remove I from U and add it to L;
        if S(R(I)) = 0 then
            Remove I from U ;
```



## Calcolo delle probabilità

In [1]:
def p_n1n2_if_true(n1, n2):
    return ((1 - errorRate) ** n1) * (errorRate ** n2)


def p_n1n2_if_false(n1, n2):
    return (errorRate ** n1) * ((1 - errorRate) ** n2)


def p_n1n2(n1, n2,):
    return selectivity * p_n1n2_if_true(n1, n2) + (1 - selectivity) * p_n1n2_if_false(n1, n2)


#la probabilità effettivamente è uno
def pr1n1n2(n1, n2):
    assert p_n1n2(n1, n2) != 0, 'Probability is not defined if P(n1, n2) = 0'
    return selectivity * p_n1n2_if_true(n1, n2) / p_n1n2(n1, n2)


#def pr0n1n2(n1, n2):
    #return 1 - pr1n1n2(n1, n2)

def pr0n1n2(n1, n2):
    assert p_n1n2(n1, n2) != 0, 'Probability is not defined if P(n1, n2) = 0'
    return (1-selectivity) * p_n1n2_if_false(n1, n2) / p_n1n2(n1, n2)


def p0(n1, n2):
    return pr1n1n2(n1, n2) * errorRate + pr0n1n2(n1, n2) * (1 - errorRate)


def p1(n1, n2):
    return pr1n1n2(n1, n2) * (1 - errorRate) + pr0n1n2(n1, n2) * errorRate

## Implementazione delle varie strategie

In [2]:
def rectangular(n1, n2):
    if n1 == M1:
        return 1
    if n2 == M2:
        return 0
    return 2

In [3]:
def majority(n1, n2, M):
    if n1 == ((M + 1) / 2):
        return 1
    if n2 == ((M + 1) / 2):
        return 0
    return 2

In [4]:
def treshold(n1,n2):
    if tresh1 < tresh2:
        print("incorrect threshold configuration... will be reversed")
        treshapp1=tresh2
        treshapp2=tresh1
    else:
        treshapp1=tresh1
        treshapp2=tresh2
        
    if treshapp1<=pr1n1n2(n1,n2):
        return 1
    else:
        if treshapp2<pr0n1n2(n1,n2):
        #if treshapp1 < pr0n1n2(n1, n2):
            return 0
        else:
            return majority(n1,n2,M)

## Calcolo di Y per ogni punto e di Y(0,0) ottimale



In [5]:
def Y(n1, n2, Y0, strategy=rectangular):
    if n1 == n2 == 0:
        return Y0
    if strategy(n1, n2) == 0:
        return Y0
    if strategy(n1, n2) == 1:
        return 0
    return min(Y0,
               1 + p1(n1, n2) * Y(n1 + 1, n2, Y0,  strategy=strategy)
               + p0(n1, n2) * Y(n1, n2 + 1, Y0, strategy=strategy))


def Y00():
    max_error = 1e-2
    max_iterations = 50
    found = False
    i = 0

    current_max = 500
    current_min = 0

    while not found:
        if i == max_iterations:
            raise Exception("Max Iterations")

        candidate_y00 = current_min + (current_max - current_min) / 2.0

     
        estimated_Y00 = 1 + p1(0, 0) * Y(1, 0,candidate_y00) + p0(0, 0) * Y(0, 1,candidate_y00)

        error = abs(estimated_Y00 - candidate_y00)
        print(
        "[i] trying Y(0, 0) = %s, estimated Y(0, 0) is %s with error: %s" % (
        str(candidate_y00), str(estimated_Y00), str(error)))

        found = error <= max_error

        if estimated_Y00 < candidate_y00:
            current_max = candidate_y00
        else:
            current_min = candidate_y00

        i += 1
    return candidate_y00

## Main e altre funzioni di supporto

In [6]:
def inizializzazione(TotalItems, selectivity):
    Item=[]
    i=0
    while(i <= TotalItems):
        while(i <= TotalItems * selectivity):
            Item.append(1)
            i+=1
        Item.append(0)
        i+=1
    random.shuffle(Item)
    Strategy={}
    c=0
    while(c < TotalItems):
        Strategy[c]=[(0,0)]
        c+=1
    #print(Strategy)
    return Item, Strategy


def min2(Y,n,Strategy):

    app=Strategy.copy()
    lower=[]
    while len(lower) < n:
        low=99999
        for key in app:
            if (Y[app[key][0]]<low):
                low=Y[app[key][0]]
                chiave=key
        lower.append(chiave)
        del app[chiave]
    return lower





# Inizio MAIN

def main():

    Item, Strategy = inizializzazione(TotalItems, selectivity)
    Y0=Y00()
    l=[]
    domande0=0
    domande1=0
    y={}
    scartati=0

    #u=Item.copy()
    app=[]

    #aggiorno elenco delle y note con i nuovi punti della strategia
    for key in Strategy:
        if(not Strategy[key] in app):
            app.append(Strategy[key])

    for ap in app:
        if(not ap[0] in y):
            y[ap[0]]=Y(ap[0][0], ap[0][1], Y0)

    fasi=0
    domande=0
    while(len(l)<K):
        fasi+=1
        I2=min2(y,K-len(l),Strategy)
        cq={}

        for i in I2:
            n1=M1-Strategy[i][0][0]
            n2=M2-Strategy[i][0][1]
            cq[i] =min(n1,n2)
            domande+=cq[i]

        for i in cq:
            c=0
            while(c<cq[i]):
                #simulo crowdsourcing
                if(random.random()>errorRate):
                    if(Item[i]==1):
                        a=Strategy[i][0][0]+1
                        b=Strategy[i][0][1]
                        domande1 += 1
                    else:
                        a = Strategy[i][0][0]
                        b = Strategy[i][0][1]+1
                        domande0 += 1
                    Strategy[i]=[(a,b)]

                else:
                    if (Item[i] == 1):
                        a = Strategy[i][0][0]
                        b = Strategy[i][0][1]+1
                        domande1 += 1
                    else:
                        a = Strategy[i][0][0]+1
                        b = Strategy[i][0][1]
                        domande0 += 1
                    Strategy[i] = [(a, b)]
                c+=1
        for i in cq:
            if rectangular(Strategy[i][0][0],Strategy[i][0][1])==1:
                l.append(i)
                del(Strategy[i])
            else:
                if rectangular(Strategy[i][0][0],Strategy[i][0][1])==0:
                    if (Item[i] == 1):
                        scartati+=1
                    del(Strategy[i])
                else:
                    if not Strategy[i][0] in y:
                        y[Strategy[i][0]]=Y(Strategy[i][0][0], Strategy[i][0][1], Y0)

    print("oggetti")
    uni=0
    for p in l:
        if(Item[p]==1):
            uni+=1
        print(p,Item[p])

    accuracy=(uni/len(l))
    avg0=domande0/(TotalItems-len(Strategy)-len(l))
    avg1=domande1/len(l)
    recall=len(l)/(scartati+len(l))

    print(""+"numero di domande medio per elementi che non soddisfano proprietà: " +str(avg0) +"\n" )
    print("" + "numero di domande medio per elementi che soddisfano proprietà: " + str(avg1)+"\n")
    print("accuracy: " + "\n\tPrecision: "+str(accuracy)+"\n\tRecall: "+ str(recall)+"\n\tScartati: "+ str(scartati))
    print("risultato ottenuto in:\n"+"\tfasi: "+str(fasi) +
          "\n"+"\tdomande:"+str(domande) )

In [7]:
import random

M1 = 10
tresh1=0.5
tresh2=0.3
M=20
M2 = 10
TotalItems=1000000
K = 15
#Y0 = 30
errorRate = 0.35
selectivity = 0.2

In [1]:
from crowPowered import *

In [2]:
main()

[i] trying Y(0, 0) = 250.0, estimated Y(0, 0) is 200.55838582223396 with error: 49.44161417776604
[i] trying Y(0, 0) = 125.0, estimated Y(0, 0) is 106.20281007258205 with error: 18.797189927417946
[i] trying Y(0, 0) = 62.5, estimated Y(0, 0) is 57.869635014363126 with error: 4.630364985636874
[i] trying Y(0, 0) = 31.25, estimated Y(0, 0) is 31.607854788593627 with error: 0.3578547885936274
[i] trying Y(0, 0) = 46.875, estimated Y(0, 0) is 45.20381910676119 with error: 1.6711808932388124
[i] trying Y(0, 0) = 39.0625, estimated Y(0, 0) is 38.45394233421315 with error: 0.6085576657868472
[i] trying Y(0, 0) = 35.15625, estimated Y(0, 0) is 35.03638692979382 with error: 0.11986307020617915
[i] trying Y(0, 0) = 33.203125, estimated Y(0, 0) is 33.323519570220746 with error: 0.12039457022074629
[i] trying Y(0, 0) = 34.1796875, estimated Y(0, 0) is 34.181340624930094 with error: 0.0016531249300939521
oggetti
2 1
12 1
10 0
5 0
25 1
24 0
27 0
37 1
42 1
40 1
63 1
68 1
80 1
83 1
85 1
numero di doma

# Script per determinare i migliori parametri da utilizzare (eseguito su Azure)

import random
import os
HEADER = "Item,K,M1,M2,selectivity,errorRate,accuracy_avg,accuracy_lower,accuracy_higher,phases_avg,phases_lower,phases_higher" \
         ",questions_avg,questions_lower,questions_higher"
OUTPUT_FILE = "analisi.csv"

if (os.path.exists(OUTPUT_FILE)):
    os.remove(OUTPUT_FILE)
    print("Removed: " + OUTPUT_FILE)
dataset = open(OUTPUT_FILE, "w")
print("Created empty file: " + OUTPUT_FILE)
dataset.write(HEADER + "\n")



tresh1=0.5
tresh2=0.3
nfasi=100
M=20

ERROR=[0.1,0.2,0.25,0.3,0.35,0.4,0.49]
SEL=[0.01,0.05,0.1,0.2,0.25,0.3]
KKK=[10,15,20,50]
M12=[5,8,10,15,20]
TOTAL=[10000,100000,1000000,10000000]

for errorRate in ERROR:
    for selectivity in SEL:
        y = {}
        for M1 in M12:
            M2=M1
            for K in KKK:
                for TotalItems in TOTAL:



                    def main():
                        .
                        .
                        .
                        return accuracy,fasi,domande
                        
                    accuracy_low=2
                    accuracy_high=-1
                    fasi_low=99999999
                    fasi_high=0
                    domande_low=999999999999
                    domande_high=0
                    count_master = 0
                    accuracy_sum=0
                    fasi_sum=0
                    domande_sum=0
                    while (count_master < nfasi):
                        accuracy,fasi,domande=main()  
                        accuracy_sum+=accuracy
                        fasi_sum+=fasi
                        domande_sum+=domande
                        if accuracy>accuracy_high:
                            accuracy_high=accuracy
                        if accuracy<accuracy_low:
                            accuracy_low=accuracy
                        if fasi>fasi_high:
                            fasi_high=fasi
                        if fasi<fasi_low:
                            fasi_low=fasi
                        if domande>domande_high:
                            domande_high=domande
                        if domande<domande_low:
                            domande_low=domande
                        count_master+=1
                        print(count_master)
                    accuracy_avg=accuracy_sum/nfasi
                    domande_avg=domande_sum/nfasi
                    fasi_avg=fasi_sum/nfasi
                    row=""+str(TotalItems)+","+str(K)+","+str(M1)+","+str(M2)+","+str(selectivity)+","+str(errorRate)+","+str(accuracy_avg)+","+str(accuracy_low)+","+str(accuracy_high)+","+str(fasi_avg)+","+str(fasi_low)+","+str(fasi_high)+","+str(domande_avg)+","+str(domande_low)+","+str(domande_high)+","
                    print(row)
                    dataset.write(row + "\n")

dataset.close()
print("File closed: " + OUTPUT_FILE)


![Esempio_Output](immagine_ris.png)

# Grafici Starplot per il confronto dei cambiamenti delle variabili

La scelta delle Starplot è per mettere a confronto in modo intituitivo, come variano gli output al variare del valore dei vari parametri in input. L'obietttivo non è tanto quello di avere metriche assolute, ma di capire i parametri di output che maggiormente risentono delle variazioni dei vari parametri di input.

## Error Rate

![Errore](immagini/Error2.png)

Al variare dell'error rate la precision e la recall scendono in particolar modo la prima. Questo perché è più facile che un oggetto 1 venga dichirato 0, il numero delle fasi e delle domande quindi diminuisce perchè seppur sbagliata si arriva prima ad una conclusione, e questo implica anche che la recall perda molto meno rispetto alla precision, proprio perchè con meno oggetti è più difficile che venga scartato un numero consistente di 1.

# Selettività

![Errore](immagini/Sele.png)

Il crescere della selettività porta a dei risultati ovviamente migliori, ma i scenari più complessi e di maggiore interesse sono quelli con una selettività piuttosto bassa. Recall e precision non ne risentono in modo netto, questo perché se di elementi che soddisfano la proprietà ce ne sono 1 o 100 per l'algoritmo non fa molta differenza da questo punto di vista (ne fa molta invece in termini di fasi  e domande), bensì è l'error Rate che ne influenza di molto il valore come visto nel plot precedente. Y00 ha un aumento esponenziale quando la selettività diminuisce al punto che negli scenari ago in un pagliaio che vedremo in seguito è impostata di default a un valore alto ottenuto durante vari esperimenti perchè per calcolarlo ci impiega moltissimo e non è molto influente ai fini del risultato quando assume valori così alti.


# Limite M della Strategia

![Errore](immagini/M.png)

M che indica il limite delle risposte yes o no per considerare un item come 1 o 0, è un parametro di cui sul paper non si parla molto e di cui è stata fatta un'attenta analisi.
Con valori molto piccoli recall e precision rischiano di crollare anche se l'error rate non è troppo alto (0.25 già è sotto lo 0.7 di precision). Da un valore pari a 10 in poi tende a stabilizzarsi sotto questo punto di vista (motivo principale per cui in tutti i test e le implementazioni successive ho assunto M1=M2=10).
Aumentando il numero di risposte di qualsiasi genere necessarie è facile da capire come il numero di fasi e domande aumenta in modo rilevante (le domande di più perchè intrinsecamente le fasi tengono conto di M dal momento che il numero di domande ad ogni fase è in funzione di quanto manca a soddisfare la strategia).
Il parametro che maggiormente cambia al crescere di M è Y00, e non solo in termini di valore ma soprattutto per il tempo di esecuzione (con 30 se non gestito correttamente nelle prime implementazioni c'era un problema di profondità della ricorsione).



# K elementi da trovare

![Errore](immagini/K.png)

K è uno dei parametri meno importanti, influsisce sul numero di domande e di fasi perchè ovviamente è necessario trovare più oggetti ma non influisce in alcun modo sugli altri aspetti

# Total Items

![Errore](immagini/Item.png)

Vale lo stesso discorso fatto per K

# Seconda Parte: Scenari alternativi

## Selettività bassa e spammer (2 probabilità)   (Scenario Di Riferimento)

Questo scenario da un punto di vista implemnetativo non si discosta molto da quello classico del paper. L'error rate non è più unico, ma ce ne sono due uno per gli elementi 0 e una per gli elementi 1.
Questo perchè si è ipotizzato che la selettività è bassissima, e che in casi come questo l'utente tende a rispondere sempre no perchè essendo quasi la totalità delle risposte corrette "no", non sbaglia quasi mai.
Quindi l'error rate per le risposte no è bassissimo( convenzionalmente messo uguale alla selettività dato che i due aspetti sono correlati) e un'error rate piuttosto alto per le risposte yes.
Dai risultati di vari test ed esecuzioni si nota come il numero delle domande e delle fasi aumenta di molto rispetto al caso in base. in particolar modo queste domande non sono ben distribuite poichè circa il 30% sono necessarie per assegnare il valore 0 e circa il 70% per quelle si.
Inoltre un aspetto che abbiamo notato è che la precision è quasi sempre alta tendente ad uno.
La spiegazione è che avendo una probabilità di errore per le risposte no praticamente nulla, le probabilità che un oggetto no con una strategia con M=10 venga dichirato si è praticamente nulla, viceversa la recall non è sempre massima perchè alcuni oggetti si vengono scartati.


In [11]:
from Scenario_ago_pagliaio1 import *


In [5]:
main()

numero di domande medio per elementi che non soddisfano proprietà: 10.010084866630926

numero di domande medio per elementi che soddisfano proprietà: 17.83

accuracy: 
	Precision: 1.0
	Recall: 1.0
	Scartati in un primo momento: 15
risultato ottenuto in:
	fasi: 4242
	domande:991390


# Risultati della configurazione (300 esecuzioni)

## Con ErrorRate 0.4

Precision	1

Recall	0,825

Fasi	3% (circa 20 volte di più dei scenari standard)

Domande	[1:15] * Total Items con una media di 9.436

## Con ErrorRate 0.35

Precision	1

Recall	0,94

Fasi	2.8% (circa 20 volte di più dei scenari standard)

Domande	[1:15] * Total Items con una media di 8.824



![Scenario2](immagini/Sc2.png)

## Variante con ricerca di tutti gli elementi

Per cercare di simulare il caso che maggiormente si avvicina a quello del caso reale nel progetto del "Codice Ratio", abbiamo introdotto la variante di cercare non k elementi che soddisfano la proprietà, ma tutti quelli che la soddisfano.
Il risultato prevede un numero elevatissimo di domande, e non garantisce che vengano prese correttamente tutte quante quando l'error rate è molto elevato

In [1]:
from Scenario_ago_pagliaio2 import *

In [2]:
main()


numero di domande medio per elementi che non soddisfano proprietà: 10.007755894479805

numero di domande medio per elementi che soddisfano proprietà: 20.75

accuracy: 
	Precision: 1.0
	Recall: 0.7524752475247525
	Scartati: 25
risultato ottenuto in:
	fasi: 1950
	domande:1001592


### Variante della precedente in cui non è consentito di scartare un oggetto che soddisfa la proprietà, variante forzata che non rispecchia uno scenario reale

In [3]:
from Scenario_ago_pagliaio2_2 import *

In [4]:
main()

numero di domande medio per elementi che non soddisfano proprietà: 10.00998900200789

numero di domande medio per elementi che soddisfano proprietà: 19.36

accuracy: 
	Precision: 1.0
	Recall: 1.0
	Scartati in un primo momento: 24
risultato ottenuto in:
	fasi: 6523
	domande:994016


# Risultati della configurazione (300 esecuzioni)

## Configurazione 1

Precision	1

Recall	0,72

Fasi	4.5% (circa 20 volte di più dei scenari standard)

Domande	[10:15] * Total Items

## Configurazione 2

Precision	1

Recall	1

Fasi	8.4% (circa 50 volte di più dei scenari standard)

Domande	[10:15] * Total Items



# Variante task con vari livelli di difficoltà e attenzione calante all'aumentare delle domande per utente


ermax=0.45
taskdiff=0.1
taskmed=0.2
tasknormal=0.7


def inizializzazione(TotalItems, selectivity):
    
    .
    
    .
    
    cont1=0
    cont2=0
    cont3=0
    while i < TotalItems:
        a=random.random()
        if (a>=(1-tasknormal)):
            #se compreso tra 0.3 e 1 nel caso 0.7
            Item2[i]=1
            cont1+=1
        else:
            if (a>taskdiff):
                #se compreso tra 0.1 e 0.3
                Item2[i]=2
                cont2 += 1
            else:
                #se minore uguale alla probabilità che sia difficile
                Item2[i]=3
                cont3 += 1
        i+=1
        
def main():

    .
    .  
    .
    errorRate = 0.30
    fasciamed=0.05
    fasciadiff=0.1
    .
    .
    .
        if errorRate1<=ermax:
            if fasi%200==0:
                #print("yes")
                n=fasi/200
                errorRate1=errorRate+n*0.005
                #print(errorRate1)

                app=errorRate1
                app2=errorRate0
                if(Item[i]==1):
                    if(Item2[Item[i]]==2):
                        errorRate1+=fasciamed
                    if(Item2[Item[i]]==3):
                        errorRate1+=fasciadiff

                   

In [7]:
from Scenario_ago_pagliaio3 import *

In [8]:
main()

task facili: 69841
task medi: 20046
task difficili: 10113
numero di domande medio per elementi che non soddisfano proprietà: 10.007726002828269

numero di domande medio per elementi che soddisfano proprietà: 18.8

accuracy: 
	Precision: 1.0
	Recall: 1.0
	Scartati in un primo momento: 5
risultato ottenuto in:
	fasi: 5780
	domande:290530


Confrontando i risultati potrebbe venir fuori un dubbio, come è possibile che con meno domande medie di questo scenario ci siano più fasi??
La risposta risiede nella natura dell'algoritmo, per ogni fase l'operazione di ricerca del minimo numero di domande necessarie produca un output minore perchè c'è meno incertezza, e quindi avremo molte fasi ognuna con un numero minore di domande.

# Risultati della configurazione (300 esecuzioni)

Precision	1

Recall	0,95

Fasi	6.5% (circa 35 volte di più dei scenari standard)

Domande	[1.23:1.62] * Total Items mediamente è intorno lo 0.3




# Variante in cui vengono inserite domande per evitare che apperentemente le risposte corrette siano tutti uguali

In questa variante vengono inserite domande in cui la risposta è uno per tenere più basso l'error rate di risposte yes.
I problemi principali di questa implementazione sono legati all'assenza di dati che ci consentono di settare opportunamente l'intervallo in cui inserire tali domande.
Dagli esperimenti fatti notiamo come all'aumentare della frequenza con cui porre tali domande,che corrisponde alla possibilità di tenere l'error rate più basso, il numero di domande e di fasi aumenta di molto. Si torna quindi al solito problema di scegliere il giusto tradeoff tra costo di domande e accuratezza delle risposte ottenute.
Nei vari test abbiamo provato a settare un approccio randomico e uno regolare, supponendo che se fosse regolare l'utente potrebbe accorgersene soprattutto se l'intervallo è ampio.
Analizzando questo scenario abbiamo inoltre ipotizzato che dal momento che le risposte non sono solo no, l'error rate delle risposte no seppur di poco va aumentato se la frequenza è molto alta. Con questa condizione ci si riavvicina allo scenario implementato dal paper, in cui le fasi e le domande reali sono mediamente costanti ma non sempre l'accuratezza in termini di precision risulta essere elevata soprattutto all'aumentare dell'error rate.


In [21]:
from Scenario_ago_pagliaio4 import *

In [22]:
main()

task facili: 70172
task medi: 19781
task difficili: 10047
numero di domande medio per elementi che non soddisfano proprietà: 11.126737338044759

numero di domande medio per elementi che soddisfano proprietà: 16.2

accuracy: 1.0
risultato ottenuto in:
	fasi: 5099
		di cui fittizzie: 4845
	domande:461495
		di cui fittizzie: 225006


# Variante in cui si gestiscono diversi tipi di utenti

In questa variante vengono individuati 3 utenti diversi, quelli normali i spammer e gli hammer.
Ognuno di loro viene contradistinto da un error rate diverso rispetto a quello degli altri.
Lo scenario di riferimento resta quello dell'ago in un pagliaio, si tiene conto dell'attenzione calante e non delle domande fittizzie per rialzarla.
In modo parametrico è possibile scegliere i vari error rate e quanti utenti ci sono per ogni tipologia, lo scenario base ipotizzato prevede un numero più elevato di utenti medi e meno utenti spammer e hammer.
Partendo da questa probabilità di errore che immaginiamo sia fornita dal servizio di crowdsourcing sulla base dello storio degli utenti, dinamicamente le probabilità di ogni singolo utente vengono aggiornate sulla base delle risposte corrette e di quelle errate. Data la selettività molto bassa abbiamo supposto che le penalità o i bonus siano relativi solo alle risposte relative agli item che soddisfano la proprietà (altrimenti gli spammer diventerebbero velocemente hammer).
Dato che chiunque può sbagliare per un qualsiasi motivo, abbiamo deciso che le risposte non sono pesate in base alla tipologia di utente, ma che ad ogni fase quando devono essere selezionati gli utenti a cui fare le domande vengono presi quelli con error rate più basso.


### Questo frammento di codice serve per determinare in ogni fase a quali utenti fare le domande

In [23]:
def min3(Utenti,n):
    app=Utenti.copy()
    while len(app) > n:
        max=0
        for key in app:
            if (app[key]>max):
                max=app[key]
                chiave=key
        del app[chiave]
    #print (app)
    return app

### Questo frammento di codice è relativo alla creazione degli utenti

In [24]:
UtSpammer=0.25
UtMedi=0.5
UtHammer=0.25
Pspammer=0.4
PNormal=0.25
Phammer=0.1

def utenti(n):
    Spammer=int(UtSpammer*n)
    Normal=int(UtMedi*n)
    Hammer=int(UtHammer*n)
    Somma=Hammer+Spammer+Normal
    if Somma<n:
        a=n-Somma
        Normal+=a
    ut={}
    c=0
    i=0
    while(i<Spammer):
        ut[c]=Pspammer
        c+=1
        i+=1
    i=0
    while(i<Normal):
        ut[c]=PNormal
        c+=1
        i+=1
    i=0
    while(i<Hammer):
        ut[c]=Phammer
        c+=1
        i+=1
    return ut


In [9]:
from Scenario_ago_pagliaio5 import *

In [10]:
main()

task facili: 69880
task medi: 20090
task difficili: 10030
numero di domande medio per elementi che non soddisfano proprietà: 10.009192627327174

numero di domande medio per elementi che soddisfano proprietà: 14.1

accuracy: 
	Precision: 1.0
	Recall: 0.9523809523809523
	Scartati: 1
risultato ottenuto in:
	fasi: 3338
	domande:215870


# Risultati della configurazione (300 esecuzioni)

Precision	1

Recall	0,95

Fasi	3.2% (circa 20 volte di più dei scenari standard)

Domande	[1.19:1.4] * Total Items mediamente è intorno lo 0.2




Questo è lo scenario con i risultati in termini di fasi e domande migliore rispetto agli altri, mantenendo comunque un elevata precision e recall. In molti casi è uno scenario che si avvicina molto anche al caso reale, è però funzione del numero di utenti per ogni categoria.